# Image Segmentation with Breast Cancer Dataset using FCN Dense Net

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
os.chdir("/kaggle/input/augmentedclear")

In [ ]:
os.listdir()
data = np.load('augmentedData_V2.npy')
label = np.load('augmentedLabel_V2.npy')

In [ ]:
dataV1 = data/2**16
labelV1 = label.astype(np.float32)

In [ ]:
print(data.shape)
print(label.shape)

In [ ]:
x_train, x_test, x_val = dataV1[:1000],dataV1[1000:1100],dataV1[1100:]
y_train, y_test, y_val = labelV1[:1000],labelV1[1000:1100],labelV1[1100:]

In [ ]:
from __future__ import absolute_import
from __future__ import print_function

import keras.models as models
from keras.optimizers import RMSprop,Adam,SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape, Permute
from keras.layers.convolutional import MaxPooling2D, UpSampling2D, Cropping2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, Conv2DTranspose, Conv3D, Lambda
from keras import backend as K
from keras.metrics import MeanIoU
from keras.models import Model
from keras.layers import Input, concatenate, Add
from keras.regularizers import l2
import json





## Functions for DensNet Model

In [ ]:
def BN_ReLU_Conv(inputs, n_filters, filter_size=3, dropout_p=0.2):
    img = BatchNormalization()(inputs)
    img = Activation('relu')(img)
    img = Conv2D(n_filters, filter_size, padding='same', kernel_initializer='he_uniform')(img)
    if dropout_p != 0.0:
        img = Dropout(dropout_p)(img)
    return img
                            
def TransitionDown(inputs, n_filters, dropout_p=0.2):
    img = BN_ReLU_Conv(inputs, n_filters, filter_size=1, dropout_p=dropout_p)
    img = MaxPooling2D((2,2))(img)
    return img

def TransitionUp(skip_connection, block_to_upsample, n_filters_keep):
    img = Conv2DTranspose(n_filters_keep, kernel_size=3, strides=2, padding='same', kernel_initializer='he_uniform')(block_to_upsample)
    img = concatenate([img, skip_connection], axis=-1)
    return img

def SoftmaxLayer(inputs, n_classes):
    img = Conv2D(n_classes, kernel_size=1, padding='same', kernel_initializer='he_uniform')(inputs)
#    l = Reshape((-1, n_classes))(l)
    img = Activation('sigmoid')(img)
    return img

## Creating Dense Net Model (FCN)

In [ ]:
def Tiramisu(
        input_shape=(512,512,1),
        n_classes = 2,
        n_filters_first_conv = 48,
        havuz_sayisi = 5,
        growth_rate = 12 ,
        block_sayisi = [5,5,5,5,5,12,5,5,5,5,5],
        dropout_p = 0.2
        ):
    
        
    #####################
    # First Convolution #
    #####################        
    inputs = Input(shape=input_shape)
    stack = Conv2D(filters=3, kernel_size=3, padding='same', kernel_initializer='he_uniform')(inputs)
    n_filters = 3*growth_rate

    #####################
    # Downsampling path #
    #####################     
    skip_connection_list = []
    
    for i in range(havuz_sayisi):
        for j in range(block_sayisi[i]):
            l = BN_ReLU_Conv(stack, growth_rate, dropout_p=dropout_p)
            stack = concatenate([stack, l])
            n_filters  += growth_rate
        skip_connection_list.append(stack)        
        stack = TransitionDown(stack, n_filters, dropout_p)
    skip_connection_list = skip_connection_list[::-1]

    
    #####################
    #    Bottleneck     #
    #####################     
    block_to_upsample=[]
    
    for j in range(block_sayisi[havuz_sayisi]):
        l = BN_ReLU_Conv(stack, growth_rate, dropout_p=dropout_p)
        block_to_upsample.append(l)
        stack = concatenate([stack,l])
    block_to_upsample = concatenate(block_to_upsample)

   
    #####################
    #  Upsampling path  #
    #####################
    for i in range(havuz_sayisi):
        n_filters_keep = growth_rate * block_sayisi[havuz_sayisi + i ]
        stack = TransitionUp(skip_connection_list[i], block_to_upsample, n_filters_keep)
        
        block_to_upsample = []
        for j in range(block_sayisi[ havuz_sayisi + i + 1 ]):
            l = BN_ReLU_Conv(stack, growth_rate, dropout_p=dropout_p)
            block_to_upsample.append(l)
            stack = concatenate([stack, l])
        block_to_upsample = concatenate(block_to_upsample)
    
    #####################
    #  Softmax          #
    #####################
    output = SoftmaxLayer(stack, n_classes)            
    model=Model(inputs = inputs, outputs = output)    
    # model.summary()
    
    return model

In [ ]:
fcDensenet = Tiramisu()
os.chdir("/kaggle/working")

model_json = fcDensenet.to_json()
with open('fcDensenet.json', 'w') as json_file:
    json_file.write(model_json)

## Dice coef for loss function


In [ ]:
def dice(y_pred, y_true):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def dice_loss(y_true, y_pred):
    return 1-dice(y_true, y_pred)

## I used early stop to avoid to overfitting.

In [ ]:
my_callbacks = [
    EarlyStopping(patience=3,monitor="loss"),
    ModelCheckpoint(filepath='DenseUnet_{epoch:02d}_{val_loss:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=True, period=1),
    #TensorBoard(log_dir='logs'),
]


optimizerA = Adam(lr=1e-4)
optimizerB = RMSprop(lr=5e-5)
optimizer = SGD(lr=3e-5)

fcDensenet.compile(loss=dice_loss,optimizer=optimizerB, metrics=MeanIoU(num_classes=2))

## Training

In [ ]:
nb_epoch = 25
batch_size = 2


history = fcDensenet.fit(x_train, y_train, 
                 batch_size=batch_size, 
                 callbacks=my_callbacks,
                 epochs=nb_epoch, verbose=1,
                 validation_data=(x_val, y_val)
                 )

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

In [ ]:
hist_df = pd.DataFrame(history.history) 
hist_csv_file = 'DenseUnet_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

## Model Testing


In [ ]:
results = fcDensenet.evaluate(x_test, y_test, batch_size=8)
print("test loss, test acc:", results)

 ## visualization of model accuracy and loss

In [ ]:
plt.plot(history.history['mean_io_u'],color="green")
plt.plot(history.history['val_mean_io_u'],color="blue")
plt.plot(history.history['loss'],color="red")
plt.plot(history.history['val_loss'],color="orange")
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val_acc','loss','val_loss'], loc='upper left')
isim_fig = "unet" + '_fig_acc.png'
plt.savefig(isim_fig)

## Prediction from test images

In [ ]:
sonuc_img = np.zeros((x_test.shape[0],512,512))
deneme = list(range(x_test.shape[0]))
for i in deneme:
    A = np.zeros((1,512,512,1))
    A[0,:,:,:] = x_test[i,:,:,:]
    B = fcDensenet.predict(A)
    C = B[:,:,:,0]
    D = C
    D = D[0,:,:]
    sonuc_img[i,:,:] = D

## Checking whether the predicted images have a zero matrix.

In [ ]:
sayac = 0
for i in range(x_test.shape[0]):
    print(np.sum(sonuc_img[i]))
    if np.sum(sonuc_img[i]) == 0:
        
        sayac += 1
        #print("boş",i)
print(sayac)

In [ ]:
print(x_test.shape)

#  visualization of predicted images

In [ ]:
label = y_test[:,:,:,0]
sonuc = sonuc_img

index = 10
plt.imshow(label[index],cmap="Greys")
plt.figure()
plt.imshow(sonuc[index],cmap="Greys")
plt.figure()
plt.imshow(x_test[index,:,:,0],cmap="Greys")

# Accuracy Test (IoU and F1 score most important for segmentation models)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
import warnings


label = label > 0.5


label = label.ravel()


sonuc = sonuc > 0.5

sonuc = sonuc.ravel()

print('Accuracy:')
print(np.mean(label == sonuc))

print('IoU Score (Jaccard Score):')
print(jaccard_score(label, sonuc))

print('F1 Score:')
print(f1_score(label, sonuc))

print('Precision:')
print(precision_score(label, sonuc))

print('Recall:')
print(recall_score(label, sonuc))

In [ ]:
label_list = []
title_list = []
print("predicted\tdata\t\tlabel")
for i in range(12):
    label_list.append(sonuc_img[i+22,:,:])
    title_list.append("predicted")
    label_list.append(x_test[i+22,:,:,0])
    title_list.append("data")
    label_list.append(y_test[i+22,:,:,0])
    title_list.append("label")
  
  

for i in range(6):
    plt.subplot(2,3,i+1)
    #plt.title(title_list[i])
    plt.imshow(label_list[i],cmap="Greys")
    plt.axis('off')
    plt.show()